## Ophyd

#### Da documentação oficial:
Ophyd cria representações de hardware em Python como objetos hierárquicos que agrupam valores relacionados do sistema de controle. Essa estrutura permite o Ophyd disponibilizar:

- Uma interface de alto nível consistente para uma ampla gama de dispositivos (usada pelo bluesky).
- Acesso direto de baixo nível ao sistema de controle para debug e desenvolvimento.

Ao apresentar uma interface uniforme, os planos experimentais podem ser independentes dos detalhes do hardware, simplificando a criação desses planos.

#### Como usar?

### Leitura de sinais Read-Only

In [ ]:
from ophyd import EpicsSignalRO

In [ ]:
i0 = EpicsSignalRO('TEST:DETECTOR:Data_RBV')

#### É possível aguardar até realizar a conexão (com timeout definido)

In [ ]:
i0.wait_for_connection(timeout=1) #Aguarda para conexão com timout definido

#### Ler o sinal com um simples dicionário, retorna o valor e um timestamp

In [ ]:
i0.read() 

#### Descrever com mais metadados, como unidade (EGU) e limites

In [ ]:
i0.describe()

### Dispositivos com permissão de escrita

In [ ]:
from ophyd import EpicsSignal

In [ ]:
data = EpicsSignal('TEST:MOTORS:m1', name='data')

In [ ]:
data.wait_for_connection()

#### Set
É possível utilizar set para realizar ações _sem bloqueio_, isto é, ao realizar data.set(10), o set iniciará o movimento, porém não irá esperar até completar

In [ ]:
data.set(0)

#### Set retorna um Status Object
Esses objetos são reponsáveis por notificar quando uma ação "lenta" foi completada. Essa ação pode ser: mover um motor, iniciar a aquisição de um detector, aguardar uma temperatura específica... Cada Status Object possui um timeout associado que pode ser configurado.

In [ ]:
status = data.set(10)

In [ ]:
status

In [ ]:
from ophyd.status import Status

status_sim = Status(timeout=10)
status_sim_2 = Status(timeout=10)

In [ ]:
import random
import threading

def done():
    status_sim.set_finished()

threading.Timer(5, done).start()

In [ ]:
print('Antes do status sim')
status_sim.wait()
print('Status sim terminou')
status_sim_2.wait()

#### Subscribe
É uma maneira de monitorar nosso sinal quando ele mudar

In [ ]:
values = [] 
def monitor(value, old_value, timestamp, **kwargs):
    values.append({'value': value, 'timestamp': timestamp})


In [ ]:
data.subscribe(monitor)

In [ ]:
data.set(-10).wait(timeout=10)
data.set(7).wait(timeout=10)
data.set(8).wait(timeout=10)

In [ ]:
values 

## Dispositivos mais comuns como EpicsMotor estão disponíveis diretamente no Ophyd:

In [ ]:
from ophyd import EpicsMotor

In [ ]:
my_motor = EpicsMotor('TEST:MOTORS:m1', name='my_motor')

## Dispositivos mais complexos
É possível unir diversos sinais e dispositivos em um único Device.

In [ ]:
from ophyd import Component as Cpt
from ophyd import Device, EpicsSignal, EpicsSignalRO, EpicsMotor, EpicsSignalWithRBV, EpicsMotor, Kind

O Component(Cpt) é usado para definir um componente de um dispositivo(nosso device). Ele serve como um descritor que conecta partes de um dispositivo ao sistema de controle.

O Component pode ser configurado com parâmetros que influenciam como o sinal é inicializado e acessado. Como por exemplo: o endereço da PV, a natureza do sinal, e outras configurações específicas dos componentes.

Considere os seguintes conjuntos de PVs:

Estágios de amostra

|Prefixo|Tipo|Descrição|
|-------|----|---------|
|TEST:MOTORS:m1|Motor|Eixo X do estágio da amostra|
|TEST:MOTORS:m2|Motor|Eixo Y do estágio da amostra|
|TEST:MOTORS:m3|Motor|Eixo Z do estágio da amostra|
|TEST:MOTORS:m4|Motor|Eixo de rotação em X do estágio da amostra|
|TEST:MOTORS:m5|Motor|Eixo de rotação em Y do estágio da amostra|
|TEST:MOTORS:m6|Motor|Eixo de rotação em Z do estágio da amostra|
|TEST:MOTORS:m7|Motor|Eixo de translação auxiliar|
|TEST:MOTORS:m8|Motor|Eixo de translação auxiliar|

Detector pontual

|Prefixo|Tipo|Descrição|
|-------|----|---------|
|TEST:DETECTOR:AcquisitionNumber|Setpoint|Sobrescreve o número de aquisições feitas até o momento.|
|TEST:DETECTOR:AcquisitionNumber_RBV|Readback|Indica o número de aquisições realizadas até o momento.|
|TEST:DETECTOR:Trigger|Setpoint|Adquire a leitura atual do detector e salva os dados internamente|
|TEST:DETECTOR:Trigger_RBV|Readback|Indica se uma nova leitura está sendo adquirida (1) ou se já foi adquirida (0).|
|TEST:DETECTOR:Data|Setpoint|Sobrescreve a leitura mais recente com inputs manuais.|
|TEST:DETECTOR:Data_RBV|Readback|Mostra a valor da última leitura do detector.|

In [ ]:
class SampleMotor(Device):
    x = Cpt(EpicsMotor, "m1", kind=Kind.hinted)
    """Eixo X do estágio da amostra."""
    y = Cpt(EpicsMotor, "m2", kind=Kind.hinted)
    """Eixo Y do estágio da amostra."""
    z = Cpt(EpicsMotor, "m3", kind=Kind.hinted)
    """Eixo Z do estágio da amostra ."""
 
    rx = Cpt(EpicsMotor, "m4")
    """Eixo de rotação em X do estágio da amostra."""
    ry = Cpt(EpicsMotor, "m5") 
    """Eixo de rotação em Y do estágio da amostra."""
    rz = Cpt(EpicsMotor, "m6")
    """Eixo de rotação em Z do estágio da amostra."""
 
    utg = Cpt(EpicsMotor, "m7")
    """Eixo de translação auxiliar."""
    uth = Cpt(EpicsMotor, "m8")
    """Eixo de translação auxiliar."""



Na definição x = Cpt(EpicsMotor, "m1", kind=Kind.hinted), o Cpt não cria imediatamente uma instância de EpicsMotor.
Na verdade, ele configura o descritor com as informações necessárias, como por exemplo: a classe EpicsMotor, o sufixo "m1", e o tipo de Kind (natureza com relação aos dados e metadados).


    

Quando a instância device de SampleMotor é criada, o dispositivo principal é inicializado, em conjunto com seus componentes.

In [ ]:
sample_stage = SampleMotor('TEST:MOTORS:', name='sample_stage')

In [ ]:
sample_stage.x

Assim, qualquer acesso subsequente a sample_stage.x retorna a mesma instância de EpicsMotor, permitindo que você controle o motor correspondente.

#### Também é possível buscar um sumário do dispositivo

In [ ]:
sample_stage.summary()

#### Para implementar o detector será necessário o uso de um Status Object diferente

In [ ]:
from ophyd.status import SubscriptionStatus

#### O SubscriptionStatus utiliza eventos Ophyd para se basear se o Status Object pode ser marcado como _finished_

In [ ]:
class SampleDetector(Device):
    acquisition_number = Cpt(EpicsSignalWithRBV, "AcquisitionNumber", kind=Kind.config)
    """Numero de aquisições realizadas até o momento."""
    trigger_signal = Cpt(EpicsSignalWithRBV, "Trigger", kind=Kind.omitted)
    """Sinal para iniciar uma nova aquisição."""
    data_signal = Cpt(EpicsSignalRO, "Data_RBV", kind=Kind.hinted)
    """Dados da última aquisição."""
 
    def trigger(self):
        super_sts = super().trigger()
 
        # Alternativa: Checar se Trigger_RBV foi para 0.
        def check_value(*, old_value, value, **kwargs):
            return (value == old_value + 1)
 
        sts = SubscriptionStatus(self.acquisition_number, check_value)
        self.trigger_signal.set(1)
 
        return super_sts & sts

Nesse exemplo, o SampleDetector implementa o método trigger (comumente utilizado em scans pelo Bluesky). Esse método é responsável por acionar o detector e reportar (via Status Object) quando o envento de trigger foi completado.

O comando super().tigger() chama o método trigger da classe base (Device), que lida com a parte padrão do acionamento (trigger). Enquanto o SubscriptionStatus verifica (monitora) se o valor de AcquitionNumber_RBV é maior que o valor anterior para notificar que a aquisição finalizou

In [ ]:
detector = SampleDetector('TEST:DETECTOR:', name='detector')

In [ ]:
detector.wait_for_connection(timeout=2)

Os tipos de Kind (natureza) influenciam como esses componentes são tratados em termos de exibição e coleta de dados, os possíveis tipos de Kind:
- hinted
- normal
- config
- omitted

As diferenças podem são mais claras ao utilizar summary, read e read_configuration

In [ ]:
detector.summary()

In [ ]:
detector.describe()

In [ ]:
detector.read()

In [ ]:
detector.read_configuration()

## Sempre preciso implementar um device novo?
## Preciso sair do zero?

## Sophys Common - https://gitlab.cnpem.br/SOL/bluesky/sophys-common

#### Sophys: Common is a collection of Bluesky and Ophyd utility code (plans, devices, and other useful stuff) for usage at LNLS/SIRIUS beamlines.
#### Sophys stands for **S**irius **Ophy**d and Bluesky utilitie**s**. As an old wise monk once said, nothing beats a cool-looking name.

#### Referências: https://blueskyproject.io/ophyd/